# auth

> User Authentication

In [ ]:
#| default_exp auth


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from todo.db import Login, users, db
from hmac import compare_digest

In [ ]:
#| export
# Auth redirect
login_redir = RedirectResponse('/login', status_code=303)

In [ ]:
#| export
# Beforeware for authentication
def before(req, sess):
    "Authentication beforeware that runs before route handlers"
    auth = req.scope['auth'] = sess.get('auth', None)
    if not auth: return login_redir
    # Set filter to only show todos for current user
    from .db import todos
    todos.xtra(name=auth)

In [ ]:
#| export
# Login routes
def login_routes(rt):
    "Define login and logout routes"
    
    @rt("/login")
    def get():
        frm = Form(
            Input(id='name', placeholder='Name'),
            Input(id='pwd', type='password', placeholder='Password'),
            Button('login'),
            action='/login', method='post')
        return Titled("Login", frm)
    
    @rt("/login")
    def post(login:Login, sess):
        if not login.name or not login.pwd: return login_redir
        try: 
            u = users[login.name]
        except NotFoundError: 
            u = users.insert(login)
        if not compare_digest(u.pwd.encode("utf-8"), login.pwd.encode("utf-8")): 
            return login_redir
        sess['auth'] = u.name
        return RedirectResponse('/', status_code=303)
    
    @rt("/logout")
    def get(sess):
        del sess['auth']
        return login_redir

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()